<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/focal_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 21kB/s 
     |████████████████████████████████| 3.8MB 46.6MB/s 
     |████████████████████████████████| 450kB 69.8MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.1.0rc0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

 84% 49.0M/58.3M [00:02<00:00, 16.7MB/s]
100% 58.3M/58.3M [00:02<00:00, 27.5MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 76.7MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 107MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 105MB/s]
 48% 25.0M/52.2M [00:01<00:01, 20.0MB/s]
100% 52.2M/52.2M [00:01<00:00, 46.9MB/s]


In [0]:
pip install keras==2.3.1

     |████████████████████████████████| 378kB 2.7MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
!kaggle competitions download -c ieee-fraud-detection

test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
pip install category_encoders

     |████████████████████████████████| 102kB 2.4MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [13]:
import pandas as pd
import numpy as np
cols=['Hidden_7_22', 'Hidden_7_32', 'Hidden_7_96', 'Hidden_7_8',
       'Hidden_3_85', 'Hidden_7_9', 'Hidden_4_5', 'Hidden_3_52',
       'Hidden_3_64', 'Hidden_3_63', 'Hidden_7_11', 'Hidden_7_33',
       'Hidden_3_28', 'Hidden_7_69', 'Hidden_7_2', 'Hidden_7_92',
       'Hidden_7_23', 'Hidden_3_99', 'Hidden_7_28', 'Hidden_3_47',
       'Hidden_3_88', 'Hidden_7_0', 'Hidden_7_34', 'Hidden_3_76',
       'Hidden_7_31', 'Hidden_7_64', 'Hidden_7_72', 'Hidden_7_70',
       'Hidden_7_10', 'Hidden_3_3', 'Hidden_4_13', 'Hidden_7_68',
       'Hidden_7_50', 'Hidden_7_57', 'Hidden_7_67', 'Hidden_7_36',
       'Hidden_3_6', 'Hidden_7_12', 'Hidden_7_91', 'Hidden_7_39',
       'Hidden_6_1', 'Hidden_3_95', 'Hidden_3_40', 'Hidden_7_83',
       'Hidden_7_29', 'Hidden_6_19', 'Hidden_4_10', 'Hidden_6_5',
       'Hidden_3_70', 'Hidden_3_86', 'Hidden_3_48', 'Hidden_3_30',
       'Hidden_7_7', 'Hidden_7_78', 'Hidden_6_10', 'Hidden_7_54',
       'Hidden_3_13', 'Hidden_7_46', 'Hidden_7_21', 'Hidden_6_3',
       'Hidden_7_73', 'Hidden_6_16', 'Hidden_3_9', 'Hidden_6_22',
       'Hidden_7_48', 'Hidden_7_66', 'Hidden_7_45', 'Hidden_6_8',
       'Hidden_7_84', 'Hidden_3_5', 'Hidden_7_18', 'Hidden_4_6',
       'Hidden_7_38', 'Hidden_6_6', 'Hidden_3_32', 'Hidden_3_83',
       'Hidden_7_14', 'Hidden_7_27', 'Hidden_5_0', 'Hidden_3_50',
       'Hidden_4_3', 'Hidden_7_65', 'Hidden_3_35', 'Hidden_7_56',
       'Hidden_3_33', 'Hidden_4_21', 'Hidden_3_77', 'Hidden_4_17',
       'Hidden_3_68', 'Hidden_3_82', 'Hidden_3_4', 'Hidden_3_75',
       'Hidden_3_20', 'Hidden_3_27', 'Hidden_3_66', 'Hidden_3_37',
       'Hidden_3_71', 'Hidden_3_89', 'Hidden_3_53', 'Hidden_3_46']
path = F"/content/gdrive/My Drive/autoenc.csv" 
selected=pd.read_csv(path,usecols=cols)
selected=selected.astype('float16')
import gc
gc.collect()

32

In [14]:
path = F"/content/gdrive/My Drive/raw1.csv" 
dumm=pd.read_csv(path)
keeps=['ProductCD','card1','card2','card3','card4','card5','card6','P_emaildomain','R_emaildomain']
trn_trans=pd.read_csv('train_transaction.csv.zip',usecols=keeps)
tst_trans=pd.read_csv('test_transaction.csv.zip',usecols=keeps)
trans=pd.concat([trn_trans,tst_trans],0)
trans=trans.reset_index(drop=True)
import gc
del([trn_trans,tst_trans])
gc.collect()
dumm=dumm.filter(regex='dum')
fnl=pd.concat([dumm,selected,trans],1)
keeps_fnl=fnl[keeps]
fnl=fnl.drop(keeps,1)
from sklearn.preprocessing import StandardScaler,MinMaxScaler
ss=StandardScaler()
mms=MinMaxScaler()
ls=list(fnl)
fnl=pd.DataFrame(mms.fit_transform(ss.fit_transform(fnl)))
fnl.columns=ls
fnl=pd.concat([fnl,keeps_fnl],1)
from sklearn.preprocessing import LabelEncoder
for col in keeps:
    le=LabelEncoder()
    fnl[col]=le.fit_transform(fnl[col].astype(str))

trn=pd.read_csv('train_transaction.csv.zip',usecols=['isFraud'])
fnl['isFraud']=trn['isFraud']
fnl['isFraud'].isna().sum()
trn=fnl.dropna()
tst=fnl.loc[fnl['isFraud'].isna()]
def rac(y_true, y_pred):
    """ ROC AUC Score.
    Approximates the Area Under Curve score, using approximation based on
    the Wilcoxon-Mann-Whitney U statistic.
    Yan, L., Dodier, R., Mozer, M. C., & Wolniewicz, R. (2003).
    Optimizing Classifier Performance via an Approximation to the Wilcoxon-Mann-Whitney Statistic.
    Measures overall performance for a full range of threshold levels.
    Arguments:
        y_pred: `Tensor`. Predicted values.
        y_true: `Tensor` . Targets (labels), a probability distribution.
    """
    with tf.name_scope("RocAucScore"):
        pos = tf.boolean_mask(y_pred, tf.cast(y_true, tf.bool))
        neg = tf.boolean_mask(y_pred, ~tf.cast(y_true, tf.bool))
        pos = tf.expand_dims(pos, 0)
        neg = tf.expand_dims(neg, 1)
        # original paper suggests performance is robust to exact parameter choice
        gamma = 0.6
        p     = 0.5
        difference = tf.zeros_like(pos * neg) + pos - neg - gamma
        masked = tf.boolean_mask(difference, difference < 0.0)
        return tf.reduce_sum(tf.pow(-masked, p))
import gc
gc.collect()
import keras.backend as K
from keras.layers import Input,Dense,BatchNormalization,Dropout,Embedding,Concatenate,Flatten
from keras.models import Model
from keras.layers import Embedding,Dot,Add,Flatten,concatenate,BatchNormalization,Input,Dense
import tensorflow as tf
from itertools import combinations
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.model_selection import KFold
from keras.optimizers import Nadam
from sklearn.metrics import roc_auc_score
from keras.regularizers import l2,l1
import keras
from sklearn.metrics import roc_auc_score
def model1(shape,er,ar,r1,r2,r3,r4):
    K.clear_session()
    inps=[]
    embs=[]
    for col in keeps:
        inp=Input((1,))
        emb=Embedding(fnl[col].max()+1,5,trainable=True,embeddings_regularizer=l1(er),activity_regularizer=l1(ar))(inp)
        inps.append(inp)
        embs.append(emb)
    x=concatenate(embs)
    x=Dense(256,activation='relu',kernel_regularizer=l1(r1))(x)
    x=BatchNormalization()(x)
    one=[Flatten()(em) for em in embs]
    second=[]
    for a,b in combinations(embs,2):
        second.append(Dot(1)([Flatten()(a),Flatten()(b)]))
    one=Add()(one)
    second=Add()(second)
    x=Flatten()(x)
    inp=Input((inp_shape,))
    inps.append(inp)
    tot=[x,(one),(second),(inp)]
    x=concatenate(tot)
    x=BatchNormalization()(x)
    x=Dense(256,activation='relu',kernel_regularizer=l1(r2))(x)
    x=BatchNormalization()(x)
    x=Dropout(0.3)(x)
    x=Dense(256,activation='relu',kernel_regularizer=l1(r3))(x)
    x=BatchNormalization()(x)
    x=Dropout(0.3)(x)
    x=Dense(256,activation='relu',kernel_regularizer=l1(r4))(x)
    x=BatchNormalization()(x)
    x=Dropout(0.3)(x)
    out=Dense(1,activation='sigmoid')(x)
    mod=Model(inputs=inps,outputs=out)
    return mod
def fl(gamma1,alpha1):
    def focal_loss(y_true, y_pred):
        gamma=gamma1
        alpha=alpha1
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        pt_1 = K.clip(pt_1, 1e-3, .999)
        pt_0 = K.clip(pt_0, 1e-3, .999)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss
def auc(y_true, y_pred):
    auc = tf.keras.metrics.AUC(y_true, y_pred)
    return auc
inp_shape=trn.shape[1]-len(keeps)
inp_shape=inp_shape-1
pre=pd.DataFrame()
    
fit=500000
batch_size=2048


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (47,50,51,53,58,59,62,63,64,65,66,69,72,73,76,77,78,79,80,81,82,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
train=trn.loc[:fit]
validation=trn.loc[fit:]
z_trn=train[keeps].values
z_val=validation[keeps].values
z_tst=tst[keeps].values
kp1=['ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'P_emaildomain',
 'R_emaildomain','isFraud']
mod=model1(trn.shape[1]-1,0.0360,0.3046,0.01,0.01,0.01,0.01)
es=EarlyStopping(monitor='val_loss',patience=100,mode='min',min_delta=0.001,restore_best_weights=True)
dc=ReduceLROnPlateau(monitor='val_loss', factor=0.8,mode='min',patience=10,min_delta=0.01)
mod.compile(optimizer=Nadam(0.0008),loss=fl(alpha1=0.75,gamma1=2.43),metrics=[tf.keras.metrics.AUC()])
history=mod.fit([z_trn[:,0],z_trn[:,1],z_trn[:,2],z_trn[:,3],z_trn[:,4],z_trn[:,5],z_trn[:,6],z_trn[:,7],z_trn[:,8]
                 ,train.drop(kp1,1).values],train['isFraud'],epochs=100,batch_size=2048,
                validation_data=([z_val[:,0],z_val[:,1],z_val[:,2],z_val[:,3],z_val[:,4],z_val[:,5],z_val[:,6],z_val[:,7],z_val[:,8]
                                  ,validation.drop(kp1,1).values],validation['isFraud']),callbacks=[es,dc])
mod.save_weights('saved.hdf5')

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 500001 samples, validate on 90540 samples
Epoch 1/100
500001/500001 [==============================] - 20s 39us/step - loss: 324.9950 - auc: 0.5918 - val_loss: 180.4807 - val_auc: 0.6543
Epoch 2/100
500001/500001 [==============================] - 17s 35us/step - loss: 164.9647 - auc: 0.7002 - val_loss: 154.3137 - val_auc: 0.7364
Epoch 3/100
500001/500001 [==============================] - 18s 35us/step - loss: 138.7838 - auc: 0.7587 - val_loss: 133.1508 - val_auc: 0.7765
Epoch 4/100
500001/500001 [==============================] - 18s 35us/step - loss: 116.1209 - auc: 0.7894 - val_loss: 109.1716 - val_auc: 0.8004
Epoch 5/100
500001/500001 [==============================] - 18s 35us/step - loss: 95.7815 - auc: 0.8091 - val_loss: 93.8003 - val_auc: 0.8161
Epoch 6/100
500001/500001 [==============================] - 17s 34us/step - loss: 78.6267 - auc: 0.8218 - val_loss: 76.8048 - val_auc: 0.8270
Epoch 7/100
500001/500001 [==============================] - 17s 34us/step - loss: 

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
path = F"/content/gdrive/My Drive/focal.hdf5" 
mod.save_weights(path)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive
